# Sampling algorithm
Implementation of Vertex sampling algorithm described in [_Web-scale information extraction with vertex_](https://ieeexplore.ieee.org/abstract/document/5767842)

## (TODO: Refactor code)

In [22]:
import requests
from lxml import html

## Compute informativeness

In [23]:
def compute_informativeness(M, values_list):
    '''
    M: number of pages in cluster
    values_list: list of values retrieved from the same xpath over each cluster page
    '''
    values_set = set(values_list)
    Ti = len(values_set)
    
    sum_F_Xi = 0
    for value in values_set:
        sum_F_Xi = sum_F_Xi + values_list.count(value)
    
    return 1 - sum_F_Xi/(M*Ti)
    

In [24]:
compute_informativeness(10, [1,2,1,1,3,5,5]) #expected: 0.825

0.825

## Get all xpath

In [25]:
def get_all_xpath(html_src):
    
    # select nodes whose children include text nodes
    XPATH_SELECTOR = "//*[child::text()]" 
    
    root = html.fromstring(html_src)
    
    tree = root.getroottree()
    
    # leaf_nodes is not properly a list of all leaf nodes. 
    # It contains nodes which are parent of text elements in the DOM
    leaf_nodes = root.xpath(XPATH_SELECTOR)
    
    xpath_list = []
    
    # extract xpath from previously selected nodes and filter out "noisy" nodes
    for leaf in leaf_nodes:
        
        xpath = tree.getpath(leaf) + "/text()"
        
        # Filtering out xpaths which extract javascript code or css stylesheet
        if  "/script" not in xpath and "/noscript" not in xpath and "/style" not in xpath:
                        
            selected_values = root.xpath(xpath)
            selected_string = ''.join(selected_values).strip()
            
            # Filtering out xpaths which extract empty strings
            if selected_string:
                xpath_list.append(xpath)

    return xpath_list    

In [26]:
r = requests.get("https://www.androidworld.it/schede/redmi-7-2/")

In [27]:
xpath_list = get_all_xpath(r.content)

In [28]:
xpath_list

['/html/head/title/text()',
 '/html/body/div/div[3]/div/header/div/div[1]/ul/li[1]/a/text()',
 '/html/body/div/div[3]/div/header/div/div[1]/ul/li[2]/a/text()',
 '/html/body/div/div[3]/div/header/div/div[1]/ul/li[3]/a/text()',
 '/html/body/div/div[3]/div/header/div/div[2]/span/a/text()',
 '/html/body/div/nav/div/ul/li[1]/a/text()',
 '/html/body/div/nav/div/ul/li[2]/a/text()',
 '/html/body/div/nav/div/ul/li[3]/a/text()',
 '/html/body/div/nav/div/ul/li[4]/a/text()',
 '/html/body/div/nav/div/ul/li[5]/a/text()',
 '/html/body/div/nav/div/ul/li[6]/a/text()',
 '/html/body/div/nav/div/ul/li[7]/a/text()',
 '/html/body/div/nav/div/ul/li[8]/a/text()',
 '/html/body/div/nav/div/ul/li[9]/a/text()',
 '/html/body/div/nav/div/ul/li[10]/a/text()',
 '/html/body/div/nav/div/ul/li[11]/a/text()',
 '/html/body/div/div[4]/div/main/div/section/article/figure/div/div[2]/a[2]/span[2]/text()',
 '/html/body/div/div[4]/div/main/div/section/article/figure/div/div[2]/a[3]/span[2]/text()',
 '/html/body/div/div[4]/div/m

In [29]:
len(xpath_list)

289

In [32]:
root = html.fromstring(r.content)
for xpath in xpath_list:
    list_of_values = root.xpath(xpath)
    string_value = ''.join(list_of_values)
    clean_value = string_value.strip()
    #print([string_value])
    print([clean_value])
    print()

['Redmi 7 (3GB) - Scheda tecnica | AndroidWorld']

['smart']

['mobile']

['android']

['AndroidWorld']

['Recensioni']

['Schede tecniche']

['Smartphone']

['Smartwatch']

['Tablet']

['App']

['Giochi']

['Guide']

['Video']

['Forum']

['📸']

['Specifiche']

['notizie']

['Correlati']

['Confronta']

['Redmi 7 (3GB)']

['Scheda Tecnica']

['Confronta']

['CPU']

['octa 1.8 GHz']

['Display']

['6,26" HD+ / 720 x 1520']

['Fotocamera']

['12 Mpx ƒ/2.2']

['Frontale']

['8 Mpx']

['RAM']

['3 GB']

['Memoria interna']

['32 / 64 GB']

['Android']

['9.0 Pie']

['Batteria']

['4000 mAh']

['è uno smartphone con sistema operativo Android di fascia bassa.']

['Redmi 7 (3GB)']

['La tecnologia del display dello smartphone è IPS LCD. Ha una diagonale di  ed una risoluzione di HD+ / 720 x 1520 e quindi un ppi di 269 ppi.']

['6,26 pollici']

['Abbiamo a che fare con una fotocamera a risoluzione di  con il supporto di un flash Singolo utile ad illuminare le foto in condizioni di scarsa luce

## Luke's functions

### Get text
Return a list of tuples ``(xpath, extracted value)`` for a page _P_

In [33]:
def get_text(html_src, xpath_list):
    
    """
    Edited: changed function signature and how extracted value is computed
    """
    
    root = html.fromstring(html_src)
    
    xpath_2_value_list = []
    
    for xpath in xpath_list:
        selected_values = root.xpath(xpath)
        selected_string = ''.join(selected_values).strip()
        xpath_2_value_list.append((xpath, selected_string))
    
    return xpath_2_value_list

In [34]:
xpath_2_values = get_text(r.content, xpath_list)

In [35]:
xpath_2_values

[('/html/head/title/text()', 'Redmi 7 (3GB) - Scheda tecnica | AndroidWorld'),
 ('/html/body/div/div[3]/div/header/div/div[1]/ul/li[1]/a/text()', 'smart'),
 ('/html/body/div/div[3]/div/header/div/div[1]/ul/li[2]/a/text()', 'mobile'),
 ('/html/body/div/div[3]/div/header/div/div[1]/ul/li[3]/a/text()', 'android'),
 ('/html/body/div/div[3]/div/header/div/div[2]/span/a/text()', 'AndroidWorld'),
 ('/html/body/div/nav/div/ul/li[1]/a/text()', 'Recensioni'),
 ('/html/body/div/nav/div/ul/li[2]/a/text()', 'Schede tecniche'),
 ('/html/body/div/nav/div/ul/li[3]/a/text()', 'Smartphone'),
 ('/html/body/div/nav/div/ul/li[4]/a/text()', 'Smartwatch'),
 ('/html/body/div/nav/div/ul/li[5]/a/text()', 'Tablet'),
 ('/html/body/div/nav/div/ul/li[6]/a/text()', 'App'),
 ('/html/body/div/nav/div/ul/li[7]/a/text()', 'Giochi'),
 ('/html/body/div/nav/div/ul/li[8]/a/text()', 'Guide'),
 ('/html/body/div/nav/div/ul/li[9]/a/text()', 'Video'),
 ('/html/body/div/nav/div/ul/li[10]/a/text()', 'Forum'),
 ('/html/body/div/nav

### Insert dict
Insert key value pair into a dictionary. This function should be deleted 

In [36]:
def insert_dict(dizionario,value):
    xpath,element=value
    if bool(dizionario.get(xpath)):
        dizionario[xpath].append(element)
    else:
        dizionario.update({xpath: [element]})

### Create data

Returns two data structures.

_First data_ is a dictionary where keys are URL and values are list of xpath.

_Second data_ is another dictionary where keys are xpath and values are list of string values extracted from xpath

_First data_ and _Second data_ are also passed as arguments to ``create_data`` (as empty dict)

Edit: changed get_text function calling instruction

In [37]:
def create_data(uri,first_data,second_data):
    r = requests.get(uri)
    lista_xpath = get_all_xpath(r.content)
    first_data.update({uri: lista_xpath})
    component_list=get_text(r.content, lista_xpath)
    for el in component_list:
        insert_dict(second_data,el)
    return first_data,second_data
  

### Data

Invokes, for each URL passed as argument, the ``create_data`` function, returning _first data_ and _second data_ (as explained previously)

In [13]:
#ciclo sulle pagine
def data(list_uri):
    first_data={}
    second_data={}
    for uri in list_uri:
        data=create_data(uri,first_data,second_data)
    #shouldn't be (first_data, second_data)?
    return data
  

### TBD

Edit: deleted ``somma`` function

In [ ]:
#obiettivo: voglio una lista di tuple di tutti gli xpath associati ad un peso w
def F(xpath):
    return len(second_data.get(xpath))  

'''
def somma(xpath):
    return F(xpath)  
'''

def I(xpath):
    return 1 - ( F(xpath) / ( len(list_uri) * len(set(second_data.get(xpath))) ) )

def w(xpath):
    return F(xpath)*I(xpath)

w_list_xpath={}

def calcola_pesi():
    for xpath in second_data:
        w_list_xpath.update({xpath: w(xpath)})

### Massimo tra pagine

In [ ]:
def massimo_tra_pagine():
    massimo_url=''
    massimo=0
    for uri in list_uri:
        somma=0
        for xpath in first_data.get(uri):
            w=w_list_xpath.get(xpath)
            if bool(w):
                somma=somma+w
        if massimo<somma:
            massimo=somma
            massimo_url=uri
      
    return massimo_url 
  

### Delete xpath

In [ ]:
def del_xpath(uri):
    for xpath in first_data.get(uri):
        if bool(w_list_xpath.get(xpath)):
            w_list_xpath.pop(xpath)

### Annotation

In [ ]:
def annotation(k=20):
    s=[]
    while bool(w_list_xpath) and len(s)<k:
        p_max=massimo_tra_pagine()
        s.append(p_max)
        del_xpath(p_max)
    return s